In [1]:
%config Inline_Backend.figure_format = "svg"
import matplotlib.pyplot as plt
import concurrent.futures
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob         

Algumas terminologias úteis:
- Sample: Cada ficheiro csv ou h5
- Evento: Cada linha das samples

Questões:
- O sinal representa nova física e o background processos já conhecidos da física de partículas? Caso contrário, qual o motivo da descrepância entre a quantidade de dados de sinal e background que utilizamos?
- Se os dados não fossem gerados, mas antes retirados diretamente de resultados de medições, como dividiríamos os dados em samples e como obteríamos as cross-sections de cada sample?

## Funções auxiliares de manipulação de dados

In [ ]:
# Variáveis auxiliares para as funções
del_cols = ["MissingET_Eta", "gen_decay1", "gen_decay2", "gen_sample", "gen_filter", "gen_decay_filter"]
btag_cols = ["Jet1_BTag", "Jet2_BTag", "Jet3_BTag", "Jet4_BTag", "Jet5_BTag"]
lepton_cols = ["Electron_Multi", "Muon_Multi"]

In [2]:
def filter_event(event):
    """
    Verifica se um evento apresenta pelo menos 1 btag e 2 leptões
    
    event -> pandas dataframe row: evento de uma sample
    return -> Bool: resultado da verificação
    """
    
    # Calcular o nº de b-tags e leptões
    b_tags = sum([event[col] for col in btag_cols])
    leptons = sum([event[col] for col in lepton_cols])
    
    return (b_tags>0 and leptons>1)


def process_df(args):
    """
    Processa uma sample:
    - Ordena as suas colunas
    - Faz cuts nos seus eventos
    - Remove colunas de dados não experimentais
    
    file -> string: file path
    
    return -> pandas dataframe: sample processada
    """
    
    # Ler a sample
    file, form = args
    if form == "csv":
        df = pd.read_csv(file)
    elif form == "h5":
        df = pd.read_hdf(file)
    
    # Ordenar das colunas da sample
    df.sort_index(axis=1, inplace=True)
        
    # Filtrar os eventos da sample
    drop_list = [i for i in range(len(df)) if not filter_event(df.loc[i])]
    print(f"Drop_Ratio '{file}': {len(drop_list)/len(df):.3f}")
    df.drop(drop_list, inplace=True)
        
    # Calcular gen_weights
    df["gen_xsec"] = df["gen_xsec"].mean()/df.shape[0]
    df.rename(columns={"gen_xsec":"gen_weights"}, inplace=True)
        
    # Remover colunas de dados não experimentais
    del_cols = [col for col in del_cols if col in df]
    df.drop(del_cols, axis=1, inplace=True)
      
    return df


def merge_files(path="", form="csv"):
    """
    Coloca todas as samples de uma diretoria num único dataframe
    Filtra os eventos de acordo com uma certa verificação
    Calcula os pesos de cada evento
    Filtra as colunas com dados não experimentais
    
    path -> string: diretoria dos dados
    form -> string: formato dos dados a importar (csv ou h5)
    
    return -> pandas dataframe: dataframe resultante da concatenação e alteração dos files
    """
            
    with concurrent.futures.ThreadPoolExecutor(2) as executor:
        # Processar paralelamente cada uma das samples
        files = glob.glob(path + "*." + form)
        args = [(file, form) for file in files]
        bkgd_data = list(tqdm(executor.map(process_df, args), total=len(files), 
                              desc="Processing samples"))
                
    # Concatenar a lista de samples
    if len(bkgd_data) > 1:
        bkgd_data = pd.concat(bkgd_data)
    else:
        bkgd_data = bkgd_data[0]
    
    return bkgd_data

## Gerar os dataframes dos dados

In [3]:
# Variáveis de controlo da criação do dataframe de background
data_path = "Data/dileptonic/"

# Criar o dataframe de background
bkgd_data = merge_files(data_path, "csv")
bkgd_data

Processing samples:   0%|          | 0/18 [00:00<?, ?it/s]

Drop_Ratio: 0.698
Drop_Ratio: 0.977


KeyboardInterrupt: 

In [ ]:
# Variáveis de controlo da criação do dataframe de sinal
data_path = "Data/FCNC/"

# Criar o dataframe de sinal
fcnc_data = merge_files(data_path, "h5", del_cols, btag_cols, lepton_cols)
fcnc_data

In [ ]:
# Guardar dados
bkgd_data.to_hdf("Data/Processed_Data.h5", key="bkgd")
fcnc_data.to_hdf("Data/Processed_Data.h5", key="fcnc")

In [ ]:
print([col for col in fcnc_data if "Tau" in col])

## Plots

Nota: Os plots estão muito diferentes dos obtidos com apenas um csv para bkgd.
Não é necessária nenhuma normalização dos gen_weights nos plot aos dados?

In [ ]:
# Importar dados pré-processados
bkgd_data = pd.read_hdf("Data/Processed_Data.h5", key="bkgd")
fcnc_data = pd.read_hdf("Data/Processed_Data.h5", key="fcnc")

In [ ]:
# Variáveis de ajuste dos plots
num_cols, bins = 5, 50

# Criar os plots
num_rows = int(np.ceil((len(list(fcnc_data.columns)) - 1) / num_cols))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(40, 60))

# Iterar e plotar cada coluna de dados
for i, column in enumerate(fcnc_data.columns):
    if fcnc_data.columns[i] != "gen_weights":
        # Definir a range do histograma
        hist_min = min(fcnc_data[column].min(), bkgd_data[column].min())
        hist_max = max(fcnc_data[column].max(), bkgd_data[column].max())
        hist_range = (hist_min, hist_max)
        
        # Obter os dados do histograma
        fcnc_hist, fcnc_bins = np.histogram(fcnc_data[column],
                                            bins=bins,
                                            range=hist_range,
                                            weights=fcnc_data["gen_weights"])
        bkgd_hist, bkgd_bins = np.histogram(bkgd_data[column],
                                            bins=bins,
                                            range=hist_range,
                                            weights=bkgd_data["gen_weights"])

        # Dar plot aos dados
        row, column = int(i/num_cols), i%num_cols
        axs[row, column].set_title(fcnc_data.columns[i])
        axs[row, column].hist(fcnc_hist, bins=fcnc_bins, 
                              label="fcnc", alpha=0.8)
        axs[row, column].hist(bkgd_hist, bins=bkgd_bins, 
                              label="bkgd", alpha=0.8)
        axs[row, column].set_yscale("log")
        axs[row, column].legend()
    
plt.show()

Significado dos gráficos:
- BTag: Ocorrência ou não de jatos a partir de muões b
- Missing: Momento de partículas não detetadas que falta para se verificar a conservação de momento na colisão
- ScalarHT: Momento total (norma) missing
- [blank]_multi: quantos [blank] existem, os plots [blank]1, [blank]2, etc descrevem cada um
- Picos em zero devido esses fenómenos não serem detetados nos respetivos eventos